# Adding Memory to an Agent

This notebook goes over adding memory to an Agent. Before going through this notebook, please walkthrough the following notebooks, as this will build on top of both of them:

- [Adding memory to an LLM Chain](adding_memory.ipynb)
- [Custom Agents](../agents/custom_agent.ipynb)

In order to add a memory to an agent we are going to the the following steps:

1. We are going to create an LLMChain with memory.
2. We are going to use that LLMChain to create a custom Agent.

For the purposes of this exercise, we are going to create a simple custom Agent that has access to a search tool and utilizes the `ConversationBufferMemory` class.

In [6]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

Notice the usage of the `chat_history` variable in the PromptTemplate, which matches up with the dynamic key name in the ConversationBufferMemory.

In [8]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

We can now construct the LLMChain, with the Memory object, and then create the agent.

In [16]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [17]:
agent_chain.run(input="How many people live in canada?")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Canada
Action: Search
Action Input: Population of Canada
Observation: The current population of Canada is 38,555,354 as of Monday, December 19, 2022, based on Worldometer elaboration of the latest United Nations data. · Canada 2020 ...
Thought: I now know the final answer
Final Answer: The current population of Canada is 38,555,354.
> Finished AgentExecutor chain.


'The current population of Canada is 38,555,354.'

To test the memory of this agent, we can ask a followup question that relies on information in the previous exchange to be answered correctly.

In [20]:
agent_chain.run(input="what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I need to find out what the national anthem of Canada is called.
Action: Search
Action Input: National anthem of Canada
Observation: After 100 years of tradition, O Canada was proclaimed Canada's national anthem in 1980. The music for O Canada was composed in 1880 by Calixa ...
Thought: I now know the final answer
Final Answer: O Canada is the national anthem of Canada.
> Finished AgentExecutor chain.


'O Canada is the national anthem of Canada.'

We can see that the agent remembered that the previous question was about Canada, and properly asked Google Search what the name of Canada's national anthem was.

For fun, let's compare this to an agent that does NOT have memory.

In [23]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_without_memory = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [24]:
agent_without_memory.run("How many people live in canada?")



> Entering new AgentExecutor chain...
Thought: I should look up the answer
Action: Search
Action Input: population of Canada
Observation: The current population of Canada is 38,555,354 as of Monday, December 19, 2022, based on Worldometer elaboration of the latest United Nations data. · Canada 2020 ...
Thought: I now know the final answer
Final Answer: The current population of Canada is 38,555,354.
> Finished AgentExecutor chain.


'The current population of Canada is 38,555,354.'

In [25]:
agent_without_memory.run("what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I should look up the answer
Action: Search
Action Input: national anthem of [country]
Observation: "Himno Nacional" ("National Anthem") · "Pátria" ("Fatherland") · "Salve, Oh Patria" ("We Salute You, Our Homeland") · "Bilady, Bilady, Bilady" ("My Country, My ...
Thought: I now know the final answer
Final Answer: The national anthem of [country] is called "Himno Nacional", "Pátria", "Salve, Oh Patria", and "Bilady, Bilady, Bilady".
> Finished AgentExecutor chain.


'The national anthem of [country] is called "Himno Nacional", "Pátria", "Salve, Oh Patria", and "Bilady, Bilady, Bilady".'